In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import tensorflow as tf
import keras
import numpy as np
import tensorflow
from keras.optimizers import SGD

import sys
sys.path.append("..")
from utils import *
from funcs import *

from mobnet import model_mobnet
from training_functions import *

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

data_path = 'data_2020/'
train_csv = data_path + 'evaluation_setup/fold1_train.csv'
val_csv = data_path + 'evaluation_setup/fold1_evaluate.csv'
feat_path = 'features/logmel128_scaled_d_dd/'
experiments = 'exp_mobnet'

if not os.path.exists(experiments):
    os.makedirs(experiments)



/scratch-local/kilic/anaconda3/envs/d20-keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/scratch-local/kilic/anaconda3/envs/d20-keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/scratch-local/kilic/anaconda3/envs/d20-keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/scratch-loca

In [2]:
# random sample data, to keep all three classes have similar number of training samples
total_csv = balance_class_data(train_csv, experiments)

num_audio_channels = 2
num_freq_bin = 128
num_time_bin = 461
num_classes = 3
max_lr = 0.1
batch_size = 32
num_epochs = 500
mixup_alpha = 0.4
sample_num = len(open(train_csv, 'r').readlines()) - 1
no = 2

data_val, y_val = load_data_2020(feat_path, val_csv, num_freq_bin, 'logmel')
y_val = keras.utils.to_categorical(y_val, num_classes)

model = model_mobnet(num_classes, input_shape=[num_freq_bin, num_time_bin, 3*num_audio_channels], num_filters=24, wd=1e-3)

model.compile(loss='categorical_crossentropy',
              optimizer = SGD(lr=max_lr,decay=0, momentum=0.9, nesterov=False),
              metrics=['accuracy']) #ori

model.summary()









Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 461, 6)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 64, 461, 6)   0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 64, 461, 6)   0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 231, 32)  1760        lambda_1[0][0]       

In [14]:

lr_scheduler = LR_WarmRestart(nbatch=np.ceil(sample_num/batch_size), Tmult=2,
                              initial_lr=max_lr, min_lr=max_lr*1e-4,
                              epochs_restart = [3.0, 7.0, 15.0, 31.0, 63.0,127.0,255.0]) 
save_path = experiments + "/model-mixup_alpha:{}-max_lr:{}-total_epochs:{}-no:{}.hdf5".format(mixup_alpha, max_lr, num_epochs, no)
checkpoint = keras.callbacks.ModelCheckpoint(save_path, monitor='val_acc', verbose=1, save_best_only=False, mode='max')
callbacks = [lr_scheduler, checkpoint]



In [15]:

train_data_generator = Generator_balanceclass_timefreqmask_nocropping_splitted(feat_path, train_csv, total_csv, experiments, num_freq_bin, 
                              batch_size=batch_size,
                              alpha=mixup_alpha, splitted_num=4)()



In [16]:
history = model.fit_generator(train_data_generator,
                              validation_data=(data_val, y_val),
                              epochs=num_epochs, 
                              verbose=1, 
                              workers=4,
                              max_queue_size = 100,
                              callbacks=callbacks,
                              steps_per_epoch=np.ceil(sample_num/batch_size)
                              ) 

Epoch 1/1
288/288 [==============================] - 46s 160ms/step - loss: 0.7199 - acc: 0.7888 - val_loss: 1.1137 - val_acc: 0.4370

LR:0.050278

Epoch 00001: saving model to exp_mobnet/model-mixup_alpha:0.4-max_lr:0.1-total_epochs:1-no:0.hdf5
